In [1]:
import numpy as np
import pandas as pd
import duckdb
from pandas_gbq import to_gbq
from pandas_gbq import read_gbq

In [5]:
# query bigquery
project_id = "projectm2-aiess"
query = "SELECT * FROM olist_brazilian_ecommerce_target.DIM_CUSTOMERS"
df_customers = read_gbq(query, project_id=project_id)
query2 = "SELECT * FROM olist_brazilian_ecommerce_target.FCT_PAYMENTS"
df_payments = read_gbq(query2, project_id=project_id)
query3 = "SELECT * FROM olist_brazilian_ecommerce_target.FCT_REVIEWS"
df_reviews = read_gbq(query3, project_id=project_id)
query4 = "SELECT * FROM olist_brazilian_ecommerce_target.DIM_GEOLOCATION"
df_geolocation = read_gbq(query4, project_id=project_id)
query5 = "SELECT * FROM olist_brazilian_ecommerce_target.FCT_ORDER_ITEMS"
df_items =  read_gbq(query5, project_id=project_id)
query6 = "SELECT * FROM olist_brazilian_ecommerce_target.DIM_DATE"
df_date = read_gbq(query6, project_id=project_id)
query7 = "SELECT * FROM olist_brazilian_ecommerce_target.DIM_ORDERS"
df_orders = read_gbq(query7, project_id=project_id)


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


<div class="alert alert-block alert-info">
Feature engineering

In [ ]:
df_orders.columns

Index(['payment_sid', 'review_sid', 'item_sid', 'customer_sid', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'TOTAL_PAYMENT'],
      dtype='object')

In [6]:
DS_orders_delivery = df_orders.copy()
DS_orders_delivery['actual_delivery_time'] = DS_orders_delivery['order_delivered_customer_date'] -  DS_orders_delivery['order_purchase_timestamp']
DS_orders_delivery['estimated delivery_time'] = DS_orders_delivery['order_estimated_delivery_date'] -DS_orders_delivery['order_purchase_timestamp']
DS_orders_delivery['actual_delivery_time_minutes'] = DS_orders_delivery['actual_delivery_time'].dt.total_seconds() / 60
DS_orders_delivery['estimated delivery_time_minutes'] = DS_orders_delivery['estimated delivery_time'].dt.total_seconds() / 60
DS_orders_delivery.columns

Index(['pk_order_sid', 'fk_customer_sid', 'fk_order_purchased_date_sid',
       'fk_order_approved_at_date_sid', 'fk_order_delivered_carrier_date_sid',
       'fk_order_delivered_customer_date_sid',
       'fk_order_estimated_delivery_date_sid', 'order_status', 'total_payment',
       'order_amt', 'freight_amt', 'total_order_amt_wf_freight', 'balance_amt',
       'payment_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'load_date', 'actual_delivery_time',
       'estimated delivery_time', 'actual_delivery_time_minutes',
       'estimated delivery_time_minutes'],
      dtype='object')

In [7]:
to_gbq(
    DS_orders_delivery,
    destination_table='olist_brazilian_ecommerce_DS.DS_orders_delivery',
    project_id='projectm2-aiess',
    if_exists='replace',
)

/home/chuhao/miniconda3/envs/dagster_meltano/lib/python3.11/site-packages/pandas_gbq/schema/pandas_to_bigquery.py:157: UserWarning: Could not determine the type of columns: actual_delivery_time, estimated delivery_time
  warnings.warn(msg)
100%|██████████| 1/1 [00:00<00:00, 5077.85it/s]


In [9]:
df_orders.columns

Index(['pk_order_sid', 'fk_customer_sid', 'fk_order_purchased_date_sid',
       'fk_order_approved_at_date_sid', 'fk_order_delivered_carrier_date_sid',
       'fk_order_delivered_customer_date_sid',
       'fk_order_estimated_delivery_date_sid', 'order_status', 'total_payment',
       'order_amt', 'freight_amt', 'total_order_amt_wf_freight', 'balance_amt',
       'payment_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'load_date'],
      dtype='object')

In [10]:
df_customers.columns

Index(['pk_customer_sid', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'load_date'],
      dtype='object')

In [11]:
df_items.columns

Index(['fk_order_sid', 'pk_order_id', 'pk_order_item_id',
       'fk_shipping_limit_date_sid', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_category_name_english',
       'product_name_length', 'product_description_length',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix',
       'seller_city', 'seller_state', 'load_date'],
      dtype='object')

In [14]:
#get buyer city + orders in df
orders_customers = pd.merge(
    df_orders,
    df_customers,
    how='left',
    left_on='fk_customer_sid',
    right_on='pk_customer_sid'
)


In [21]:
print("orders_customers columns:", orders_customers.columns)

orders_customers columns: Index(['pk_order_sid', 'fk_customer_sid', 'fk_order_purchased_date_sid',
       'fk_order_approved_at_date_sid', 'fk_order_delivered_carrier_date_sid',
       'fk_order_delivered_customer_date_sid',
       'fk_order_estimated_delivery_date_sid', 'order_status', 'total_payment',
       'order_amt', 'freight_amt', 'total_order_amt_wf_freight', 'balance_amt',
       'payment_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'load_date_x', 'pk_customer_sid',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'load_date_y'],
      dtype='object')


In [22]:
print("df_items columns:", df_items.columns)

df_items columns: Index(['fk_order_sid', 'pk_order_id', 'pk_order_item_id',
       'fk_shipping_limit_date_sid', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_category_name_english',
       'product_name_length', 'product_description_length',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix',
       'seller_city', 'seller_state', 'load_date'],
      dtype='object')


In [18]:
#get buyer city + orders in df
orders_customers = pd.merge(
    df_orders,
    df_customers,
    how='left',
    left_on='fk_customer_sid',
    right_on='pk_customer_sid'
)

#if prices are needed
orders_full = pd.merge(
    orders_customers,
    df_items[['fk_order_sid', 'pk_order_item_id', 'price']],
    how='left',
    left_on='pk_order_sid',
    right_on='fk_order_sid'
)

In [20]:
orders_full.columns

Index(['pk_order_sid', 'fk_customer_sid', 'fk_order_purchased_date_sid',
       'fk_order_approved_at_date_sid', 'fk_order_delivered_carrier_date_sid',
       'fk_order_delivered_customer_date_sid',
       'fk_order_estimated_delivery_date_sid', 'order_status', 'total_payment',
       'order_amt', 'freight_amt', 'total_order_amt_wf_freight', 'balance_amt',
       'payment_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'load_date_x', 'pk_customer_sid',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'load_date_y', 'fk_order_sid', 'pk_order_item_id',
       'price'],
      dtype='object')

In [23]:
orders_full['profit'] = orders_full['total_payment'] - orders_full['price']

In [24]:
orders_full.columns

Index(['pk_order_sid', 'fk_customer_sid', 'fk_order_purchased_date_sid',
       'fk_order_approved_at_date_sid', 'fk_order_delivered_carrier_date_sid',
       'fk_order_delivered_customer_date_sid',
       'fk_order_estimated_delivery_date_sid', 'order_status', 'total_payment',
       'order_amt', 'freight_amt', 'total_order_amt_wf_freight', 'balance_amt',
       'payment_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'load_date_x', 'pk_customer_sid',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'load_date_y', 'fk_order_sid', 'pk_order_item_id',
       'price', 'profit'],
      dtype='object')

In [25]:
to_gbq(
    orders_full,
    destination_table='olist_brazilian_ecommerce_DS.DS_orders_full_profits',
    project_id='projectm2-aiess',
    if_exists='replace',
)

100%|██████████| 1/1 [00:00<00:00, 10866.07it/s]


In [26]:
DS_land_geolocation = df_geolocation.copy()

DS_land_geolocation = DS_land_geolocation[
    (DS_land_geolocation['geolocation_lat'].between(-34, 5)) &
    (DS_land_geolocation['geolocation_lng'].between(-74, -34))
]



In [27]:
to_gbq(
    DS_land_geolocation,
    destination_table='olist_brazilian_ecommerce_DS.DS_land_geolocation',
    project_id='projectm2-aiess',
    if_exists='replace',
)

100%|██████████| 1/1 [00:00<00:00, 7738.57it/s]
